In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from logo import Logo, Oldlogo
import logo_comparison, deep_ocr
from logo_comparison import *
from deep_ocr import *

import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

## Make empty database for Oldlogos

In [54]:
database = pd.DataFrame(columns=['path', 
                                 'name',
                                 'image',
                                 'text', 
                                 'primary', 
                                 'secondary', 
                                 'rgb_df', 
                                 'contours', 
                                 'contour_count', 
                                 'contour_area', 
                                 'contour_points', 
                                 'image_with_contours'])

## Load in database from storage

In [55]:
# database = pd.read_csv('database.csv')

In [56]:
# Loading the data
data_dfs = dict()

## Create Logo objects from scratch

In [57]:
group = np.arange(1, 7)

In [58]:
# for i in group:

#     applicant_loc = f'/Users/aaronrasin/Desktop/Logo/Logos/applicant_{i}'
#     applicant_logo_names = os.listdir(applicant_loc)
#     applicant_logos = list()

#     previous_loc = f'/Users/aaronrasin/Desktop/Logo/Logos/previous_{i}'
#     previous_logo_names = os.listdir(previous_loc)
#     previous_logos = list()
    
#     print(f"Initializing applicant logos (Test Set {i})")
#     for applicant in tqdm(applicant_logo_names):
#         if applicant != '.DS_Store':
#             applicant_logos.append(Logo(applicant_loc + '/' + applicant))
            
#     print(f"Initializing previous logos (Test Set {i})")
#     for previous in tqdm(previous_logo_names):
#         if previous != '.DS_Store':
#             previous_logos.append(Logo(previous_loc + '/' + previous))
#             row = store_logo(previous_logos[-1])
#             database = database.append(row, ignore_index=True)

## Create applicant Logos

In [59]:
for i in group:
    print(f"Initializing applicant logos (Test Set {i})")

    applicant_loc = f'/Users/aaronrasin/Desktop/Logo/Logos/applicant_{i}'
    applicant_logo_names = os.listdir(applicant_loc)
    
    for applicant in tqdm(applicant_logo_names):
        if applicant != '.DS_Store':
            applicant_logos.append(Logo(applicant_loc + '/' + applicant))

Initializing applicant logos (Test Set 1)


100%|███████████████████████████████████████████| 24/24 [01:15<00:00,  3.13s/it]


Initializing applicant logos (Test Set 2)


100%|█████████████████████████████████████████████| 9/9 [00:12<00:00,  1.40s/it]


Initializing applicant logos (Test Set 3)


...

100%|███████████████████████████████████████████| 17/17 [00:37<00:00,  2.19s/it]


Initializing applicant logos (Test Set 5)


100%|███████████████████████████████████████████| 34/34 [03:57<00:00,  6.98s/it]


Initializing applicant logos (Test Set 6)


100%|███████████████████████████████████████████| 16/16 [00:47<00:00,  2.94s/it]


## Create previous Logos (and store them)

In [60]:
for i in group:        
    print(f"Initializing previous logos (Test Set {i})")
    
    previous_loc = f'/Users/aaronrasin/Desktop/Logo/Logos/previous_{i}'
    previous_logo_names = os.listdir(previous_loc)
    
    for previous in tqdm(previous_logo_names):
        if previous != '.DS_Store':
            previous_logos.append(Logo(previous_loc + '/' + previous))
            row = store_logo(previous_logos[-1])
            database = database.append(row, ignore_index=True)
            
database.to_csv('database.csv', index=False)

Initializing previous logos (Test Set 1)


100%|███████████████████████████████████████████| 35/35 [01:28<00:00,  2.54s/it]


Initializing previous logos (Test Set 2)


100%|█████████████████████████████████████████████| 7/7 [00:14<00:00,  2.03s/it]


Initializing previous logos (Test Set 3)


...

100%|███████████████████████████████████████████| 18/18 [00:34<00:00,  1.90s/it]


Initializing previous logos (Test Set 5)


100%|███████████████████████████████████████████| 40/40 [05:36<00:00,  8.42s/it]


Initializing previous logos (Test Set 6)


100%|███████████████████████████████████████████| 16/16 [00:52<00:00,  3.29s/it]


## Load in Oldlogos

In [55]:
database

In [7]:
previous_oldlogos = list()

In [24]:
for i in group:
    print(f"Uploading previous logos (Test Set {i})")
    names = database['name'].unique()
    
    for oldname in tqdm(previous_logo_names):
        if oldname in names:
            previous_logos.append(Oldlogo(oldname, database))

Uploading previous logos (Test Set 1)


100%|██████████████████████████████████████████| 16/16 [00:00<00:00, 613.38it/s]


Uploading previous logos (Test Set 2)


100%|██████████████████████████████████████████| 16/16 [00:00<00:00, 331.23it/s]


Uploading previous logos (Test Set 3)


...

100%|██████████████████████████████████████████| 16/16 [00:00<00:00, 664.73it/s]


Uploading previous logos (Test Set 5)


100%|██████████████████████████████████████████| 16/16 [00:00<00:00, 760.79it/s]


Uploading previous logos (Test Set 6)


100%|█████████████████████████████████████████| 16/16 [00:00<00:00, 1502.76it/s]


In [27]:
len(previous_logos)

90

## Compare and scale logos

In [18]:
    # Comparing applicants and previous logos
    df = compare_logos(applicant_logos, previous_logos)
    
    # Scale the similarity scores (SSIM, Color, and Template)
    df = scale_scores(df, features = ['SSIM', 'Color Similarity Score', 'Template Matching'])
    
    data_dfs[f'df{i}'] = df

Initializing applicant logos (Test Set 5)


100%|███████████████████████████████████████████| 34/34 [02:20<00:00,  4.13s/it]


Initializing previous logos (Test Set 5)


100%|███████████████████████████████████████████| 40/40 [03:53<00:00,  5.84s/it]


NameError: name 'df' is not defined

Need an Oldlogo class for logos that have already been analyzed and stored. 

In [35]:
def store_logo(logoA):
    row = {
        'path': logoA.orig_path,
        'name': logoA.name,
        'image': logoA.image,
        'text': logoA.text,
        'primary': logoA.primary,
        'secondary': logoA.secondary,
        'rgb_df': logoA.rgb_df,
        'contours': logoA.contours,
        'contour_count': logoA.contour_count,
        'contour_area': logoA.contour_area,
        'contour_points': logoA.contour_points,
        'image_with_contours': logoA.image_with_contours
    }
    return row